In [2]:
import pandas as pd
import numpy as np

def topsis(data_path, criteria_weights, benefit_criteria, cost_criteria):
    """
    Implémente la méthode TOPSIS pour la prise de décision multi-critères.

    Args:
        data_path (str): Le chemin vers le fichier Excel contenant les données.
                         La première colonne doit contenir les noms des alternatives,
                         les colonnes suivantes les évaluations pour chaque critère.
        criteria_weights (dict): Un dictionnaire où les clés sont les noms des critères
                                 et les valeurs sont leurs poids.
        benefit_criteria (list): Une liste des noms des critères à maximiser.
        cost_criteria (list): Une liste des noms des critères à minimiser.

    Returns:
        pandas.DataFrame: Un DataFrame contenant les alternatives, leurs CC, et leur classement.
    """

    try:
        # Lecture simplifiée pour uniquement les fichiers Excel
        df = pd.read_excel(data_path)

        # Nettoyage des noms de colonnes
        df.columns = df.columns.str.replace('\xa0', ' ').str.strip()

    except FileNotFoundError:
        print(f"Erreur: Le fichier Excel '{data_path}' est introuvable. Veuillez vérifier le chemin.")
        return None
    except Exception as e:
        print(f"Une erreur est survenue lors de la lecture du fichier: {e}")
        return None

    # Assurez-vous que les colonnes des critères sont numériques et gérez les NaN
    criteria_columns = list(criteria_weights.keys())
    for col in criteria_columns:
        if col not in df.columns:
            print(f"Erreur: Le critère '{col}' spécifié dans les poids n'existe pas dans le fichier.")
            print(f"Les colonnes disponibles sont : {df.columns.tolist()}")
            return None
        # Convertir en numérique, les erreurs deviendront NaN
        df[col] = pd.to_numeric(df[col], errors='coerce')

    # Supprime les lignes où il y a des NaN dans N'IMPORTE QUELLE colonne de critère
    df.dropna(subset=criteria_columns, inplace=True)

    if df.empty:
        print("Erreur: Le DataFrame est vide après la suppression des lignes avec des valeurs manquantes. Impossible de continuer.")
        return None

    # Réinitialise l'index après une possible suppression de lignes
    df.reset_index(drop=True, inplace=True)

    # Capture des alternatives et création du DataFrame d'évaluations
    alternatives = df.iloc[:, 0].tolist()
    evaluations_df = df[criteria_columns]

    # Étape 1(a): Normalisation
    sum_sq_evaluations = np.sqrt(np.sum(evaluations_df**2, axis=0))
    sum_sq_evaluations[sum_sq_evaluations == 0] = 1 # Éviter la division par zéro
    normalized_matrix = evaluations_df / sum_sq_evaluations

    # Étape 1(b): Pondération
    weighted_matrix = normalized_matrix.copy()
    for col, weight in criteria_weights.items():
        weighted_matrix[col] = weighted_matrix[col] * weight

    # Étape 2: Solution idéale positive (PIS)
    pis = []
    for col in criteria_columns:
        if col in benefit_criteria:
            pis.append(weighted_matrix[col].max())
        else: # On assume que si ce n'est pas un bénéfice, c'est un coût
            pis.append(weighted_matrix[col].min())
    pis = np.array(pis)

    # Étape 3: Solution idéale négative (NIS)
    nis = []
    for col in criteria_columns:
        if col in benefit_criteria:
            nis.append(weighted_matrix[col].min())
        else: # On assume que si ce n'est pas un bénéfice, c'est un coût
            nis.append(weighted_matrix[col].max())
    nis = np.array(nis)

    # Étape 4(a): Calcul des distances
    d_plus = np.sqrt(np.sum((weighted_matrix - pis)**2, axis=1))
    d_minus = np.sqrt(np.sum((weighted_matrix - nis)**2, axis=1))

    # Étape 4(b): Calcul du coefficient de proximité (CC)
    denominator = d_plus + d_minus
    cc_values = np.divide(d_minus.to_numpy(), denominator.to_numpy(), out=np.zeros_like(d_minus.to_numpy(), dtype=float), where=denominator.to_numpy()!=0)

    # Étape 5: Création du DataFrame final et classement
    results_df = pd.DataFrame({
        'Alternative': alternatives,
        'Coefficient de Proximité (CC)': cc_values
    })
    results_df['Classement'] = results_df['Coefficient de Proximité (CC)'].rank(ascending=False, method='min')
    results_df = results_df.sort_values(by='Coefficient de Proximité (CC)', ascending=False).reset_index(drop=True)

    return results_df

# --- Paramètres à modifier par l'utilisateur ---

# Chemin vers votre fichier Excel
excel_file_path = r'C:\Users\Dell\Downloads\Classement.xlsx'

# Poids des critères (vous pouvez ajuster un poids de 0.0001 si vous voulez que la somme soit exactement 1)
poids_criteres = {
    'Débit cumulée': 0.3500, # Ajusté pour que la somme soit 1.0
    'Volume de gaz': 0.3500,
    'Distance au CPF': 0.1742,
    'THP moyen': 0.0868,
    "Production d'eau": 0.0390
}

# Critères à maximiser (bénéfice)
criteres_benefice = [
    'Débit cumulée',
    'Volume de gaz',
    'THP moyen'
]

# Critères à minimiser (coût)
criteres_cout = [
    'Distance au CPF',
    "Production d'eau"
]

# --- Exécution de la fonction TOPSIS ---
if __name__ == "__main__":
    topsis_results = topsis(excel_file_path, poids_criteres, criteres_benefice, criteres_cout)

    if topsis_results is not None:
        print("--- Résultats TOPSIS ---")
        print(topsis_results)

        print("\n--- Meilleure Alternative ---")
        print(f"L'alternative la mieux classée est : {topsis_results.iloc[0]['Alternative']} "
              f"avec un CC de {topsis_results.iloc[0]['Coefficient de Proximité (CC)']:.4f}")

        # Exportation des résultats en fichier CSV
        try:
            output_csv_path = 'topsis_ranking.csv'
            # Utilise le point comme séparateur décimal pour une compatibilité maximale
            topsis_results.to_csv(output_csv_path, index=False, sep=';') 
            print(f"\n✅ Les résultats ont été sauvegardés avec succès dans le fichier '{output_csv_path}'")
        except Exception as e:
            print(f"\n❌ Erreur lors de la sauvegarde du fichier CSV: {e}")

--- Résultats TOPSIS ---
    Alternative  Coefficient de Proximité (CC)  Classement
0      chmp1-15                       0.936910         1.0
1       chmp4-4                       0.811130         2.0
2       chmp1-3                       0.761900         3.0
3      chmp1-24                       0.714134         4.0
4       chmp5-4                       0.708370         5.0
..          ...                            ...         ...
138    chmp8-1B                       0.230730       139.0
139    chmp8_W3                       0.227363       140.0
140   chmp9-102                       0.225693       141.0
141    chmp9-15                       0.219975       142.0
142   chmp7-NW3                       0.212043       143.0

[143 rows x 3 columns]

--- Meilleure Alternative ---
L'alternative la mieux classée est : chmp1-15 avec un CC de 0.9369

✅ Les résultats ont été sauvegardés avec succès dans le fichier 'topsis_ranking.csv'
